# Modeling study with MARS and neural networks

Guillaume Thibault 

`Using Python`


Aims to study the relationship between 11 quality of life indicators and the value of a residence

In [10]:
!python --version

Python 3.8.10


In [ ]:
!pip install pandas numpy pyearth plotly scikit-learn

In [ ]:
!pip install git+https://github.com/scikit-learn-contrib/py-earth@v0.2dev

In [8]:
import pandas as pd 
import numpy as np # for data manipulation
from sklearn.linear_model import LinearRegression 
from pyearth import Earth 
import plotly.graph_objects as go
import plotly.express as px 

### Data
The file of 506 observations is divided into 2 groups
* GROUP = M for Model development (405 observations: 80% of the observations) the M data are highlighted (green) and constitute a filter; the statistical modeling is based on these data (405 obs.) see Tools...selections conditions...edit
* GROUP = T for Testing the model (101 observations: 20% of observations) to include this group in an analysis, edit the filter


INDICATORS
* X1 CRIM : CRIMe Rate Per Capita by town
* X2 NOX : Nitric OXide concentration (parts per 10 million)
* X3 AGE: Proportion of owner occupied units built prior to 1940
* X4 DIS: Weighted DIStances to five Boston employment centers
* X5 RM: Average number of RooMs per dwelling
* X6 LSTAT: % of the Lower STATus of the population
* X7 RAD : Index of accessibility to RADical highways
* X8 CHAS : CHASrles river dummy variable (1 if census tract bounds the river; 0 otherwise)
* X9 INDUS : Proportion of non-retail INDUStrial business acres per town
* X10 TAX : Full value property TAX rate per $10,000
* X11 PT : Pupil-Teacher ratio by town
* RLZ : Proportion of Residential Land Zoned for lots over 25,000 sq.ft.
 available in the file but will not be used


RESPONSE
* Y_MV : Median Value of owner occupied homes (in $1000's)

The models will be developed with the M group of 406 observations.

In [9]:
df = pd.read_csv('data.csv')

df.head()

,CT,X1_CRIM,X2_NOX,X3_AGE,X4_DIS,X5_RM,X6_LSTAT,X7_RAD,X8_ CHAS,X9_NDUS,X10_TAX,X11_PT,Y_MV,GROUP,X12_RLZ
0,1,0.00632,0.538,65.2,4.0900,6.575,4.98,1,0,2.31,296,15.3,24.0,M,18.0
1,2,0.02731,0.469,78.9,4.9671,6.421,9.14,2,0,7.07,242,17.8,21.6,T,0.0
2,3,0.02729,0.469,61.1,4.9671,7.185,4.03,2,0,7.07,242,17.8,34.7,M,0.0
3,4,0.03237,0.458,45.8,6.0622,6.998,2.94,3,0,2.18,222,18.7,33.4,M,0.0
4,5,0.06905,0.458,54.2,6.0622,7.147,5.33,3,0,2.18,222,18.7,36.2,M,0.0


In [18]:
# Create a scatter plot
fig = px.scatter(df, x=df['X6_LSTAT'], y=df['Y_MV'], 
                 opacity=0.8, color_discrete_sequence=['black'])

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

# Set figure title
fig.update_layout(title_text="Scatter Plot")

# Update marker size
fig.update_traces(marker=dict(size=3))

fig.show()


In [24]:
# --- Select variables to use in the two models --- 
# Note, we need X to be a 2D array, hence reshape
X=df['X6_LSTAT'].values.reshape(-1,1)
y=df['Y_MV'].values

# --- Define and fit the two models ---
model1 = LinearRegression() 
model2 = Earth(max_terms=500, max_degree=1) # note, terms in brackets are the hyperparameters 

LR = model1.fit(X, y)
MARS = model2.fit(X, y)

# --- Print model summary ---
# LR
print("Simple Linear Regression Model")
print("--------------------------------------")
print("Intercept: ", LR.intercept_)
print("Slope: ", LR.coef_)

print("")
print("")

# MARS
print(MARS.summary())

Simple Linear Regression Model
--------------------------------------
Intercept:  34.5538408793831
Slope:  [-0.95004935]


Earth Model
-------------------------------------
Basis Function  Pruned  Coefficient  
-------------------------------------
(Intercept)     No      11.9581      
h(x0-6.12)      Yes     None         
h(6.12-x0)      No      4.11806      
h(x0-22.74)     Yes     None         
h(22.74-x0)     No      0.87503      
x0              Yes     None         
-------------------------------------
MSE: 26.4084, GCV: 27.0460, RSQ: 0.6872, GRSQ: 0.6809


In [27]:

# ----- Create data for Linear Regression line -----
# Create 20 evenly spaced points from smallest X to largest X
x_range = np.linspace(X.min(), X.max(), 20) 

# Predict y values for our set of X values
y_range = model1.predict(x_range.reshape(-1, 1))


# ----- Create data for MARS model line -----
# Select a few points including the 3 major ones: min, max and hinge
x_mars = np.array([X.min(), 5, 10, 15, 20, 25, 30, 35, X.max()])

# Predict y values for our set of X values
y_mars = model2.predict(x_mars.reshape(-1, 1))


# ----- Create a scatter plot -----
fig = px.scatter(df, x=df['X6_LSTAT'], y=df['Y_MV'], 
                 opacity=0.8, color_discrete_sequence=['black'])

# Add a best-fit line
fig.add_traces(go.Scatter(x=x_range, y=y_range, name='Linear Regression', line=dict(color='limegreen')))
fig.add_traces(go.Scatter(x=x_mars, y=y_mars, name='MARS', line=dict(color='red')))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black')

# Set figure title
fig.update_layout(title_text="Linear Regression vs. MARS")

# Update marker size
fig.update_traces(marker=dict(size=3))

fig.show()

#### Two variables

In [28]:
# Create a 3D scatter plot
fig = px.scatter_3d(df, x=df['X6_LSTAT'], y=df['X3_AGE'], z=df['Y_MV'], 
                 opacity=0.8, color_discrete_sequence=['black'])

# Set figure title
fig.update_layout(title_text="Scatter 3D Plot",
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black', 
                                          gridcolor='lightgrey')))

# Update marker size
fig.update_traces(marker=dict(size=3))

fig.show()

In [29]:

# ----- Select variables that we want to use in a model -----
# Note, X in this case is already a 2D array, hence no reshape
X=df[['X6_LSTAT','X3_AGE']]
y=df['Y_MV'].values

# ----- Define and fit the two models -----
model1 = LinearRegression()
model2 = Earth()

LR = model1.fit(X, y)
MARS = model2.fit(X, y)

# ----- Print model summary -----
# LR
print("Simple Linear Regression Model")
print("--------------------------------------")
print("Intercept: ", LR.intercept_)
print("Slope: ", LR.coef_)

print("")

print("")

# MARS
print(MARS.summary())


Simple Linear Regression Model
--------------------------------------
Intercept:  33.2227605317929
Slope:  [-1.03206856  0.03454434]


Earth Model
----------------------------------------
Basis Function     Pruned  Coefficient  
----------------------------------------
(Intercept)        No      -59.2357     
h(X6_LSTAT-6.12)   No      4.25796      
h(6.12-X6_LSTAT)   Yes     None         
h(X6_LSTAT-22.74)  No      -4.21402     
h(22.74-X6_LSTAT)  No      5.23979      
X3_AGE             Yes     None         
h(X3_AGE-54.4)     Yes     None         
h(54.4-X3_AGE)     No      -0.109485    
h(X3_AGE-77.7)     Yes     None         
h(77.7-X3_AGE)     Yes     None         
----------------------------------------
MSE: 24.9447, GCV: 26.0656, RSQ: 0.7045, GRSQ: 0.6925


In [31]:
X

,X6_LSTAT,X3_AGE
0,4.98,65.2
1,9.14,78.9
2,4.03,61.1
3,2.94,45.8
4,5.33,54.2
...,...,...
501,9.67,69.1
502,9.08,76.7
503,5.64,91.0
504,6.48,89.3


In [30]:

# Increments between points in a meshgrid
mesh_size = 10

# Identify min and max values for input variables
x_min, x_max = X['X6_LSTAT'].min(), X['X6_LSTAT'].max()
y_min, y_max = X['X3_AGE'].min(), X['X3_AGE'].max()

# Return evenly spaced values based on a range between min and max
xrange = np.arange(x_min, x_max, mesh_size)
yrange = np.arange(y_min, y_max, mesh_size)

# Create a meshgrid
xx, yy = np.meshgrid(xrange, yrange)

# Predict using LR model
pred_LR = model1.predict(np.c_[xx.ravel(), yy.ravel()])
pred_LR = pred_LR.reshape(xx.shape)

# Predict using MARS model
pred_MARS = model2.predict(np.c_[xx.ravel(), yy.ravel()])
pred_MARS = pred_MARS.reshape(xx.shape)

# Note, .ravel() flattens the array to a 1D array,
# then np.c_ takes elements from flattened xx and yy arrays and puts them together,
# this creates the right shape required for model input

# prediction array that is created by the model output is a 1D array,
# we need to reshape it to be the same shape as xx or yy to be able to display it on a graph

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning:

X does not have valid feature names, but LinearRegression was fitted with feature names



In [32]:

# Create a 3D scatter plot with predictions
fig = px.scatter_3d(df, x=df['X6_LSTAT'], y=df['X3_AGE'], z=df['Y_MV'], 
                 opacity=0.8, color_discrete_sequence=['black'])

# Set figure title and colors
fig.update_layout(title_text="Scatter 3D Plot with Linear Regression Prediction Surface",
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black', 
                                          gridcolor='lightgrey')))
# Update marker size
fig.update_traces(marker=dict(size=3))

# Add prediction plane
fig.add_traces(go.Surface(x=xrange, y=yrange, z=pred_LR, name='LR'))

fig.show()

In [33]:

# Create a 3D scatter plot with predictions
fig = px.scatter_3d(df, x=df['X6_LSTAT'], y=df['X3_AGE'], z=df['Y_MV'], 
                 opacity=0.8, color_discrete_sequence=['black'])

# Set figure title and colors
fig.update_layout(title_text="Scatter 3D Plot with MARS Prediction Surface",
                  scene = dict(xaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'),
                               yaxis=dict(backgroundcolor='white',
                                          color='black',
                                          gridcolor='lightgrey'
                                          ),
                               zaxis=dict(backgroundcolor='white',
                                          color='black', 
                                          gridcolor='lightgrey')))
# Update marker size
fig.update_traces(marker=dict(size=3))

# Add prediction plane
fig.add_traces(go.Surface(x=xrange, y=yrange, z=pred_MARS, name='MARS', ))
                          #colorscale=px.colors.sequential.Viridis))

fig.show()